# 🤖 LLM模型下载与评估工具

这个notebook可以帮你从HuggingFace下载模型并进行全面评估。

**评估维度：**
1. 困惑度 (Perplexity) - 30分
2. 推理速度 (Inference Speed) - 30分  
3. 生成质量 (Generation Quality) - 25分
4. 模型效率 (Model Efficiency) - 15分

**推荐运行环境：**
- Google Colab (免费T4 GPU)
- 本地Jupyter (需要GPU)

---

## 📦 Step 1: 安装依赖包

首次运行需要安装所需的库

In [1]:
%%capture
# 安装必要的依赖包
!pip install torch transformers datasets accelerate bitsandbytes pandas numpy tqdm

## 🔧 Step 2: 导入库和定义评估器类

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import time
import json
import numpy as np
import pandas as pd
from typing import Dict, List
import warnings
warnings.filterwarnings('ignore')

print("✅ 库导入成功！")
print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

/home/jovyan/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jovyan/.local/lib/python3.13/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


✅ 库导入成功！
PyTorch版本: 2.9.0+cu128
CUDA可用: True
GPU: NVIDIA H100 80GB HBM3 MIG 1g.20gb


In [3]:
class LLMEvaluator:
    """LLM模型评估器"""
    
    def __init__(self, model_name: str, device: str = "auto"):
        self.model_name = model_name
        self.device = self._setup_device(device)
        self.model = None
        self.tokenizer = None
        
    def _setup_device(self, device: str) -> str:
        if device == "auto":
            return "cuda" if torch.cuda.is_available() else "cpu"
        return device
    
    def download_and_load_model(self, load_in_8bit: bool = False, load_in_4bit: bool = False):
        """从HuggingFace下载并加载模型"""
        print(f"\n{'='*60}")
        print(f"正在下载模型: {self.model_name}")
        print(f"目标设备: {self.device}")
        print(f"{'='*60}\n")
        
        try:
            # 加载tokenizer
            print("📥 加载Tokenizer...")
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.model_name,
                trust_remote_code=True
            )
            
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            print("✅ Tokenizer加载完成")
            
            # 加载模型
            print("📥 加载模型...")
            model_kwargs = {
                "trust_remote_code": True,
                "torch_dtype": torch.float16 if self.device == "cuda" else torch.float32,
            }
            
            if load_in_8bit:
                model_kwargs["load_in_8bit"] = True
                print("⚙️  使用8bit量化")
            elif load_in_4bit:
                model_kwargs["load_in_4bit"] = True
                print("⚙️  使用4bit量化")
            elif self.device == "cuda":
                model_kwargs["device_map"] = "auto"
            
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                **model_kwargs
            )
            
            if self.device == "cpu" and not (load_in_8bit or load_in_4bit):
                self.model = self.model.to(self.device)
            
            print("✅ 模型加载成功！\n")
            
        except Exception as e:
            print(f"❌ 模型加载失败: {e}")
            raise
    
    def evaluate_perplexity(self, texts: List[str] = None, num_samples: int = 50) -> float:
        """评估困惑度"""
        print("\n[1/4] 📊 评估困惑度 (Perplexity)...")
        
        if texts is None:
            try:
                print("   加载WikiText-2数据集...")
                dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
                texts = [item['text'] for item in dataset if len(item['text']) > 50][:num_samples]
            except:
                texts = [
                    "The quick brown fox jumps over the lazy dog.",
                    "Artificial intelligence is transforming the world.",
                    "Machine learning models require large amounts of data."
                ]
        
        total_loss = 0
        total_tokens = 0
        
        self.model.eval()
        with torch.no_grad():
            for i, text in enumerate(texts):
                if (i + 1) % 10 == 0:
                    print(f"   处理中: {i+1}/{len(texts)}")
                
                encodings = self.tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
                input_ids = encodings.input_ids.to(self.device)
                
                outputs = self.model(input_ids, labels=input_ids)
                total_loss += outputs.loss.item() * input_ids.size(1)
                total_tokens += input_ids.size(1)
        
        perplexity = torch.exp(torch.tensor(total_loss / total_tokens)).item()
        
        # 评分
        if perplexity < 20:
            rating = "🌟 优秀"
        elif perplexity < 50:
            rating = "😊 良好"
        elif perplexity < 100:
            rating = "😐 中等"
        else:
            rating = "😞 需改进"
        
        print(f"   ✅ 困惑度: {perplexity:.2f} {rating}")
        return perplexity
    
    def evaluate_generation_quality(self, prompts: List[str] = None) -> Dict:
        """评估生成质量"""
        print("\n[2/4] 📝 评估生成质量...")
        
        if prompts is None:
            prompts = [
                "Explain quantum computing in simple terms:",
                "Write a short story about a robot:",
                "What is the capital of France?",
                "Translate 'Hello' to Spanish:",
                "List three benefits of exercise:"
            ]
        
        results = []
        generation_times = []
        
        for i, prompt in enumerate(prompts, 1):
            print(f"   生成中 {i}/{len(prompts)}: {prompt[:40]}...")
            
            start_time = time.time()
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
            
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=100,
                    temperature=0.7,
                    top_p=0.9,
                    do_sample=True,
                    pad_token_id=self.tokenizer.pad_token_id
                )
            
            generation_time = time.time() - start_time
            generation_times.append(generation_time)
            
            generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            results.append({
                "prompt": prompt,
                "response": generated_text[len(prompt):].strip(),
                "time": generation_time
            })
        
        avg_time = np.mean(generation_times)
        print(f"\n   ✅ 平均生成时间: {avg_time:.2f}秒")
        
        return {
            "avg_generation_time": avg_time,
            "samples": results
        }
    
    def evaluate_model_size(self) -> Dict:
        """评估模型大小"""
        print("\n[3/4] 📏 评估模型规模...")
        
        total_params = sum(p.numel() for p in self.model.parameters())
        trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        
        param_size = sum(p.nelement() * p.element_size() for p in self.model.parameters())
        buffer_size = sum(b.nelement() * b.element_size() for b in self.model.buffers())
        size_mb = (param_size + buffer_size) / 1024**2
        
        print(f"   参数量: {total_params:,} ({total_params/1e9:.2f}B)")
        print(f"   模型大小: {size_mb:.2f} MB ({size_mb/1024:.2f} GB)")
        
        return {
            "total_params": total_params,
            "trainable_params": trainable_params,
            "size_mb": size_mb
        }
    
    def evaluate_inference_speed(self, num_runs: int = 10) -> Dict:
        """评估推理速度"""
        print(f"\n[4/4] ⚡ 评估推理速度 (运行{num_runs}次)...")
        
        test_input = "The future of artificial intelligence is"
        inputs = self.tokenizer(test_input, return_tensors="pt").to(self.device)
        
        # 预热
        with torch.no_grad():
            self.model.generate(**inputs, max_new_tokens=50)
        
        times = []
        tokens_generated = []
        
        for i in range(num_runs):
            start = time.time()
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=50,
                    do_sample=False
                )
            elapsed = time.time() - start
            times.append(elapsed)
            tokens_generated.append(outputs.shape[1] - inputs.input_ids.shape[1])
        
        avg_time = np.mean(times)
        avg_tokens = np.mean(tokens_generated)
        tokens_per_second = avg_tokens / avg_time
        
        # 评级
        if tokens_per_second > 50:
            rating = "🚀 优秀"
        elif tokens_per_second > 30:
            rating = "😊 良好"
        elif tokens_per_second > 10:
            rating = "😐 中等"
        else:
            rating = "🐌 较慢"
        
        print(f"   ✅ 推理速度: {tokens_per_second:.2f} tokens/s {rating}")
        
        return {
            "avg_inference_time": avg_time,
            "tokens_per_second": tokens_per_second
        }
    
    def _calculate_overall_score(self, results: Dict) -> float:
        """计算综合评分"""
        score = 0
        
        # 困惑度评分 (30分)
        if results.get("perplexity"):
            ppl = results["perplexity"]
            if ppl < 20:
                score += 30
            elif ppl < 50:
                score += 25
            elif ppl < 100:
                score += 20
            elif ppl < 200:
                score += 15
            else:
                score += 10
        
        # 推理速度评分 (30分)
        if results.get("inference_speed"):
            tps = results["inference_speed"]["tokens_per_second"]
            if tps > 50:
                score += 30
            elif tps > 30:
                score += 25
            elif tps > 20:
                score += 20
            elif tps > 10:
                score += 15
            else:
                score += 10
        
        # 生成质量评分 (25分)
        if results.get("generation"):
            avg_time = results["generation"]["avg_generation_time"]
            if avg_time < 2:
                score += 25
            elif avg_time < 5:
                score += 20
            elif avg_time < 10:
                score += 15
            else:
                score += 10
        
        # 模型效率评分 (15分)
        if results.get("model_size"):
            params_b = results["model_size"]["total_params"] / 1e9
            if 1 <= params_b <= 10:
                score += 15
            elif params_b < 1:
                score += 12
            elif params_b <= 20:
                score += 10
            else:
                score += 5
        
        return score
    
    def run_comprehensive_evaluation(self, show_samples: bool = True) -> Dict:
        """运行完整评估"""
        print("\n" + "="*70)
        print(f"🔍 开始评估模型: {self.model_name}")
        print("="*70)
        
        results = {}
        
        # 1. 困惑度
        try:
            results["perplexity"] = self.evaluate_perplexity()
        except Exception as e:
            print(f"   ⚠️  困惑度评估失败: {e}")
            results["perplexity"] = None
        
        # 2. 生成质量
        try:
            results["generation"] = self.evaluate_generation_quality()
        except Exception as e:
            print(f"   ⚠️  生成质量评估失败: {e}")
            results["generation"] = None
        
        # 3. 模型规模
        try:
            results["model_size"] = self.evaluate_model_size()
        except Exception as e:
            print(f"   ⚠️  模型规模评估失败: {e}")
            results["model_size"] = None
        
        # 4. 推理速度
        try:
            results["inference_speed"] = self.evaluate_inference_speed()
        except Exception as e:
            print(f"   ⚠️  推理速度评估失败: {e}")
            results["inference_speed"] = None
        
        # 综合评分
        score = self._calculate_overall_score(results)
        results["overall_score"] = score
        
        # 显示结果
        print("\n" + "="*70)
        print("📊 评估结果汇总")
        print("="*70)
        
        if results.get("perplexity"):
            print(f"困惑度:      {results['perplexity']:.2f}")
        if results.get("inference_speed"):
            print(f"推理速度:    {results['inference_speed']['tokens_per_second']:.2f} tokens/s")
        if results.get("generation"):
            print(f"生成时间:    {results['generation']['avg_generation_time']:.2f}s")
        if results.get("model_size"):
            print(f"参数量:      {results['model_size']['total_params']/1e9:.2f}B")
        
        # 评分等级
        if score >= 90:
            grade = "🏆 优秀"
        elif score >= 75:
            grade = "🥈 良好"
        elif score >= 60:
            grade = "🥉 中等"
        else:
            grade = "📝 需改进"
        
        print("="*70)
        print(f"综合评分:    {score:.1f}/100 {grade}")
        print("="*70)
        
        # 显示生成样例
        if show_samples and results.get("generation") and results["generation"].get("samples"):
            print("\n📝 生成样例:")
            for i, sample in enumerate(results["generation"]["samples"][:2], 1):
                print(f"\n[样例 {i}]")
                print(f"Q: {sample['prompt']}")
                print(f"A: {sample['response'][:150]}...")
        
        return results

print("✅ LLMEvaluator类定义完成！")

✅ LLMEvaluator类定义完成！


## ⚙️ Step 3: 配置要评估的模型

修改下面的 `MODEL_NAME` 来评估不同的模型

In [12]:
# 🔧 配置区域 - 修改这里来评估不同的模型

# 选择要评估的模型（从下面的列表中选择或输入自定义模型）
MODEL_NAME = "zoomerwork/model_gemma"

# 量化选项（如果内存不足，设置为True）
USE_4BIT = False  # 4bit量化，大幅减少内存占用
USE_8BIT = False  # 8bit量化

# 评估选项
SHOW_GENERATION_SAMPLES = True  # 是否显示生成样例

print("配置完成！")
print(f"模型: {MODEL_NAME}")
print(f"4bit量化: {USE_4BIT}")
print(f"8bit量化: {USE_8BIT}")

配置完成！
模型: zoomerwork/model_gemma
4bit量化: False
8bit量化: False


### 📋 推荐模型列表

**小型模型** (适合CPU或小GPU，1-3B参数):
- `TinyLlama/TinyLlama-1.1B-Chat-v1.0` ⭐ 推荐新手
- `microsoft/phi-2` (2.7B)
- `stabilityai/stablelm-2-1_6b` (1.6B)

**中型模型** (需要GPU，7-13B参数):
- `mistralai/Mistral-7B-Instruct-v0.2`
- `google/gemma-7b-it`
- `meta-llama/Llama-2-7b-chat-hf` (需要HF token)

**大型模型** (需要大显存或量化，30B+):
- `meta-llama/Llama-2-70b-chat-hf` (建议USE_4BIT=True)

---

## 🚀 Step 4: 加载模型

这一步会从HuggingFace下载模型（首次运行可能需要几分钟）

In [13]:
# 创建评估器
evaluator = LLMEvaluator(MODEL_NAME, device="auto")

# 下载并加载模型
evaluator.download_and_load_model(
    load_in_4bit=USE_4BIT,
    load_in_8bit=USE_8BIT
)


正在下载模型: zoomerwork/model_gemma
目标设备: cuda

📥 加载Tokenizer...
✅ Tokenizer加载完成
📥 加载模型...
✅ 模型加载成功！



## 📊 Step 5: 运行完整评估

这将运行所有4个评估维度并给出综合评分

In [14]:
# 运行完整评估
results = evaluator.run_comprehensive_evaluation(show_samples=SHOW_GENERATION_SAMPLES)


🔍 开始评估模型: zoomerwork/model_gemma

[1/4] 📊 评估困惑度 (Perplexity)...
   加载WikiText-2数据集...
   处理中: 10/50
   处理中: 20/50
   处理中: 30/50
   处理中: 40/50
   处理中: 50/50
   ✅ 困惑度: 25.26 😊 良好

[2/4] 📝 评估生成质量...
   生成中 1/5: Explain quantum computing in simple term...
   生成中 2/5: Write a short story about a robot:...
   生成中 3/5: What is the capital of France?...
   生成中 4/5: Translate 'Hello' to Spanish:...
   生成中 5/5: List three benefits of exercise:...

   ✅ 平均生成时间: 0.58秒

[3/4] 📏 评估模型规模...
   参数量: 2,614,341,888 (2.61B)
   模型大小: 4986.46 MB (4.87 GB)

[4/4] ⚡ 评估推理速度 (运行10次)...
   ✅ 推理速度: 67.03 tokens/s 🚀 优秀

📊 评估结果汇总
困惑度:      25.26
推理速度:    67.03 tokens/s
生成时间:    0.58s
参数量:      2.61B
综合评分:    95.0/100 🏆 优秀

📝 生成样例:

[样例 1]
Q: Explain quantum computing in simple terms:
A: ...

[样例 2]
Q: Write a short story about a robot:
A: The robot, with its metal limbs and circuits, was created with one purpose: to serve. It had no emotions, no thoughts, no dreams. It only knew its tas...


## 💾 Step 6: 保存结果

In [15]:
# 保存详细结果到JSON
output_filename = f"evaluation_{MODEL_NAME.replace('/', '_')}.json"

with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False, default=str)

print(f"\n💾 结果已保存到: {output_filename}")


💾 结果已保存到: evaluation_zoomerwork_model_gemma.json


## 🔬 可选：单独测试各个维度

如果你想单独测试某个维度，可以运行下面的代码块

In [8]:
# # 单独测试困惑度
# custom_texts = [
#     "人工智能正在改变世界。",
#     "Machine learning is a subset of artificial intelligence.",
#     "深度学习需要大量的数据和计算资源。"
# ]

# ppl = evaluator.evaluate_perplexity(texts=custom_texts)

In [9]:
# # 单独测试生成质量（使用自定义问题）
# custom_prompts = [
#     "用简单的语言解释什么是机器学习：",
#     "写一个关于未来的短故事：",
#     "What are the benefits of learning Python?"
# ]

# gen_results = evaluator.evaluate_generation_quality(prompts=custom_prompts)

# # 显示详细结果
# for i, sample in enumerate(gen_results["samples"], 1):
#     print(f"\n{'='*60}")
#     print(f"问题 {i}: {sample['prompt']}")
#     print(f"{'='*60}")
#     print(f"回答: {sample['response']}")
#     print(f"耗时: {sample['time']:.2f}秒")

In [10]:
# # 单独测试推理速度（更多运行次数以获得更准确的结果）
# speed_results = evaluator.evaluate_inference_speed(num_runs=20)

# print(f"\n平均推理时间: {speed_results['avg_inference_time']:.3f}秒")
# print(f"推理速度: {speed_results['tokens_per_second']:.2f} tokens/s")

## 📈 比较多个模型

如果你想比较多个模型，可以运行下面的代码

In [11]:
# # 定义要比较的模型列表
# models_to_compare = [
#     "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
#     # "microsoft/phi-2",
#     # 添加更多模型...
# ]

# comparison_results = []

# for model_name in models_to_compare:
#     print(f"\n\n{'#'*70}")
#     print(f"正在评估: {model_name}")
#     print(f"{'#'*70}\n")
    
#     try:
#         # 创建新的评估器
#         eval = LLMEvaluator(model_name, device="auto")
#         eval.download_and_load_model(load_in_4bit=True)  # 使用4bit节省内存
        
#         # 运行评估
#         result = eval.run_comprehensive_evaluation(show_samples=False)
        
#         # 添加到结果列表
#         comparison_results.append({
#             "模型名称": model_name.split('/')[-1],
#             "完整名称": model_name,
#             "综合评分": result.get("overall_score", 0),
#             "困惑度": result.get("perplexity", "N/A"),
#             "推理速度": result.get("inference_speed", {}).get("tokens_per_second", "N/A"),
#             "参数量(B)": result.get("model_size", {}).get("total_params", 0) / 1e9,
#         })
        
#         # 清理内存
#         del eval
#         torch.cuda.empty_cache() if torch.cuda.is_available() else None
        
#     except Exception as e:
#         print(f"❌ 评估失败: {e}")
#         comparison_results.append({
#             "模型名称": model_name.split('/')[-1],
#             "错误": str(e)
#         })

# # 创建对比表格
# df = pd.DataFrame(comparison_results)
# df = df.sort_values("综合评分", ascending=False)

# print("\n\n" + "="*100)
# print("📊 模型对比结果")
# print("="*100)
# print(df.to_string(index=False))
# print("="*100)

# # 保存对比结果
# df.to_csv("model_comparison.csv", index=False, encoding='utf-8-sig')
# print("\n💾 对比结果已保存到: model_comparison.csv")

## 🎯 评分标准说明

### 综合评分 (0-100分)

| 分数范围 | 等级 | 说明 |
|---------|------|------|
| 90-100 | 🏆 优秀 | 生产级别，适合实际部署 |
| 75-89 | 🥈 良好 | 适合大多数应用场景 |
| 60-74 | 🥉 中等 | 特定场景可用 |
| <60 | 📝 需改进 | 实验性质 |

### 各维度评分

**困惑度 (30分)**
- < 20: 优秀 (30分)
- 20-50: 良好 (25分)
- 50-100: 中等 (20分)
- \> 100: 需改进 (10-15分)

**推理速度 (30分)**
- \> 50 tokens/s: 优秀 (30分)
- 30-50 tokens/s: 良好 (25分)
- 10-30 tokens/s: 中等 (15-20分)
- < 10 tokens/s: 较慢 (10分)

**生成质量 (25分)**
- 基于平均生成时间和响应质量

**模型效率 (15分)**
- 1B-10B参数: 最佳平衡 (15分)
- < 1B: 较小 (12分)
- 10B-20B: 较大 (10分)
- \> 20B: 很大 (5分)

---

## 📚 使用建议

1. **首次运行**: 建议从小模型开始（如TinyLlama）
2. **内存不足**: 启用4bit量化 (`USE_4BIT=True`)
3. **GPU加速**: 确保安装了CUDA版本的PyTorch
4. **下载加速**: 可以设置HuggingFace镜像

```python
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
```

---

## ❓ 常见问题

**Q: 下载速度慢怎么办？**
- A: 设置HuggingFace镜像或使用代理

**Q: 内存溢出 (OOM) 怎么办？**
- A: 启用4bit量化，或选择更小的模型

**Q: 如何评估私有模型？**
- A: 确保已登录HuggingFace: `huggingface-cli login`

**Q: 评估结果如何解读？**
- A: 参考上面的评分标准，综合考虑各项指标

---

## 📝 License

MIT License - 可自由使用和修改
